In [50]:
import pandas as pd
import numpy as np

In [52]:
# open match file
df = pd.read_csv('../match-data/nwsl/2025-05-24_Angel_City__vs__Racing_Louisville.csv')

In [54]:
df.head()

id  eventId  typeId  periodId  timeMin  timeSec  \
0  2818672333        1      34        16        0        0   
1  2818672929        1      34        16        0        0   
2  2818692387        2      32         1        0        0   
3  2818692393        2      32         1        0        0   
4  2818692401        3       1         1        0        0   

                contestantId  outcome     x     y                 timeStamp  \
0    6hbfgdczbi3yg4sdpqgtu6s        1   0.0   0.0  2025-05-25T01:19:13.029Z   
1  dtucewmt49th1p4selfutmnmc        1   0.0   0.0  2025-05-25T01:20:01.094Z   
2    6hbfgdczbi3yg4sdpqgtu6s        1   0.0   0.0  2025-05-25T02:05:26.440Z   
3  dtucewmt49th1p4selfutmnmc        1   0.0   0.0  2025-05-25T02:05:26.440Z   
4    6hbfgdczbi3yg4sdpqgtu6s        1  49.8  50.0  2025-05-25T02:05:26.441Z   

           lastModified                                          qualifier  \
0  2025-05-25T02:06:34Z  [{'id': 5748890963, 'qualifierId': 130, 'value...   
1  2025-05-25T02:17:40Z  [{'id': 5748894195, 'qualifierId': 227, 'value...   
2  2025-05-25T02:05:26Z  [{'id': 5748989733, 'qualifierId': 127, 'value...   
3  2025-05-25T02:05:28Z  [{'id': 5748989751, 'qualifierId': 127, 'value...   
4  2025-05-25T02:56:28Z  [{'id': 5748989791, 'qualifierId': 213, 'value...   

                    playerId playerName  keyPass  assist              Squad  \
0                        NaN        NaN      NaN     NaN         Angel City   
1                        NaN        NaN      NaN     NaN  Racing Louisville   
2                        NaN        NaN      NaN     NaN         Angel City   
3                        NaN        NaN      NaN     NaN  Racing Louisville   
4  as9gd28disfeeiiqxn7gjvg2c  K. Fuller      NaN     NaN         Angel City   

  Position  
0     Home  
1     Away  
2     Home  
3     Away  
4     Home

In [56]:
def infer_carries(df):
    df = df.sort_values(by=['playerName', 'periodId', 'timeMin', 'timeSec'])
    carry_rows = []

    for i in range(len(df) - 1):
        current = df.iloc[i]
        next_event = df.iloc[i + 1]

        if current['playerName'] == next_event['playerName']:
            time_diff = (next_event['timeMin'] * 60 + next_event['timeSec']) - (current['timeMin'] * 60 + current['timeSec'])
            dist = np.sqrt((next_event['x'] - current['x'])**2 + (next_event['y'] - current['y'])**2)

            if 0 < time_diff <= 10 and dist > 2:
                carry_rows.append({
                    'playerName': current['playerName'],
                    'x': current['x'],
                    'y': current['y'],
                    'Next_X': next_event['x'],
                    'Next_Y': next_event['y'],
                    'timeMin': current['timeMin'],
                    'timeSec': current['timeSec'],
                    'contestantId': current['contestantId']
                })

    return pd.DataFrame(carry_rows)

df_carries = infer_carries(df)

# get progressive carries
df_carries['beginning'] = np.sqrt((105 - df_carries['x'])**2 + (52.5 - df_carries['y'])**2)
df_carries['end'] = np.sqrt((105 - df_carries['Next_X'])**2 + (52.5 - df_carries['Next_Y'])**2)
df_carries['progressive'] = (df_carries['end'] / df_carries['beginning']) < 0.75

In [58]:
df_carries

playerName     x     y  Next_X  Next_Y  timeMin  timeSec  \
0    A. Anderson  14.7  28.8    14.8    41.0       10       13   
1    A. Anderson  13.3  36.9     8.2    50.4       20       48   
2    A. Anderson   5.0  48.1    37.2    -1.2       29       12   
3    A. Anderson   8.5  46.0    16.5    48.8       39       25   
4    A. Anderson   5.3  49.4    28.8   101.4       49       33   
..           ...   ...   ...     ...     ...      ...      ...   
376     T. Flint  15.6  26.5    37.1   101.8       80       54   
377     T. Flint  24.1  20.6    22.4    25.3       92       15   
378     T. Flint  73.2  91.4    56.1    73.0       95       40   
379     T. Flint  83.3  89.4    73.5    84.8       96       47   
380     T. Flint  73.5  84.8   101.4    92.4       96       48   

                  contestantId   beginning        end  progressive  
0      6hbfgdczbi3yg4sdpqgtu6s   93.358342  90.930138        False  
1      6hbfgdczbi3yg4sdpqgtu6s   93.017471  96.822776        False  
2      6hbfgdczbi3yg4sdpqgtu6s  100.096753  86.490057        False  
3      6hbfgdczbi3yg4sdpqgtu6s   96.718664  88.577311        False  
4      6hbfgdczbi3yg4sdpqgtu6s   99.748183  90.540875        False  
..                         ...         ...        ...          ...  
376  dtucewmt49th1p4selfutmnmc   93.104028  83.910071        False  
377  dtucewmt49th1p4selfutmnmc   86.962176  86.963211        False  
378  dtucewmt49th1p4selfutmnmc   50.243905  53.023202        False  
379  dtucewmt49th1p4selfutmnmc   42.807710  45.116959        False  
380  dtucewmt49th1p4selfutmnmc   45.116959  40.062077        False  

[381 rows x 11 columns]

In [60]:
print(df_carries[df_carries['progressive'] != False]['playerName'].value_counts())

playerName
M. A. Vignola    5
S. DeMelo        4
K. Fischer       4
A. Thompson      4
C. Petersen      3
G. Thompson      3
R. Tiernan       3
K. Fuller        3
M. Reid          3
M. Hodge         2
M. Moriya        2
M. Hammond       2
A. Kennedy       1
S. Weber         1
S. Mattice       1
K. Zelem         1
L. Milliet       1
K. O'Kane        1
J. Sonis         1
J. Bloomer       1
E. Sears         1
C. Press         1
T. Flint         1
Name: count, dtype: int64
